In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.optimize as spo
import statsmodels.api as sm
from statsmodels.graphics.dotplots import dot_plot
from scipy.interpolate import interp1d
import seaborn as sns
params = {'figure.figsize': (10,7),
          'axes.labelsize': 18,
          'font.size': 18,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'legend.fontsize': 16}
sns.set(palette='Set2', rc=params)
sns.set_style('white')

### puzzle 1 ###

```
17 16 15 14 13
18  5  4  3 12
19  6  1  2 11           <-- spiralled grid, 1 is at (0,0), 13 is at (+2, +2)
20  7  8  9 10
21 22 23 24 25 26
```

In [2]:
def grid_size(j):
    return (2*j + 1)**2
def perimeter_size(j):
    if j == 0:
        return 1
    else:
        return 8*j
def perimeter_side(j):
    return 2*j + 1
def first_num_in_perimeter(j):
    if j == 0:
        return 1
    else:
        return (2*j - 1)**2 + 1
def perimeter_origin(j):
    if j == 0:
        return (0, 0)
    else:
        return (+j, -(j-1))
def manhattan_distance_to_origin(a, b): 
    #https://en.wikipedia.org/wiki/Taxicab_geometry
    return abs(a) + abs(b)

In [3]:
cell = 312051
jcell = int((np.sqrt(cell) - 1)/2) + 1
print(jcell, grid_size(jcell), first_num_in_perimeter(jcell))

279 312481 310250


Wrapping around each perimeter:

```
 7  6  5  4  3
 8  3  2  1  2
 9  4  0  0  1
10  5  6  7  0
11 12 13 14 15  0
```

In [4]:
def NE(j):
    Lj = perimeter_side(j)
    if j == 0:
        return 0
    else:
        return Lj - 2
def NW(j):
    Lj = perimeter_side(j)
    if j == 0:
        return 0
    else:
        return 2*Lj - 3
def SW(j):
    Lj = perimeter_side(j)
    if j == 0:
        return 0
    else:
        return 3*Lj - 4
def SE(j):
    if j == 0:
        return 0
    else:
        return perimeter_size(j) - 1

In [5]:
for j in (0, 1, 2, 3):
    print(NE(j), NW(j), SW(j), SE(j))

0 0 0 0
1 3 5 7
3 7 11 15
5 11 17 23


In [6]:
offset = first_num_in_perimeter(jcell)
perim_loc = cell - offset
print (perim_loc)
print(NE(jcell), NW(jcell), SW(jcell), SE(jcell))

1801
557 1115 1673 2231


1801 is between SW and SE, so it's on the south portion of the perimeter.

In [7]:
def NEpos(j):
    return (+j,+j)
def NWpos(j):
    return (-j,+j)
def SWpos(j):
    return (-j, -j)
def SEpos(j):
    return (+j, -j)

In [8]:
SWpos(jcell)

(-279, -279)

In [9]:
xcell = SWpos(jcell)[0] + (perim_loc - SW(jcell))
ycell = SWpos(jcell)[1]
print(xcell, ycell)

-151 -279


In [10]:
distance = manhattan_distance_to_origin(xcell, ycell)
print(distance)

430


### puzzle 2 ###

Same spiral storage pattern, but the numbers in each cell come from the numbers filled beforehand.

* The center is filled with 1
* Each perimeter is filled in order with the sum of all adjacent filled-in values (including diagonally adjacent
* What is the first cell value that is larger than the input ("cell") of the first puzzle?

```
147 142 133 122  59
304   5   4   2  57
330  10   1   1  54
351  11  23  25  26
362 747 806 ...
```

It's not worth trying to do math here, just fill the silly thing.

In [37]:
size = 11
A = np.zeros([size, size])
def ridx(y):
    return size//2 - y
def cidx(x):
    return size//2 + x
A[ridx(0), cidx(0)] = 1

In [38]:
def fill_value(x, y):
    row = ridx(y); col=cidx(x)
    A[row, col] = sum(A[row+roff, col+coff] for roff in (-1, 0, +1) for coff in (-1, 0, +1))

In [39]:
def fill_perimeter(p):
    x0, y0 = perimeter_origin(p)
    L = perimeter_side(p)
    w = L//2
    # east side fill
    x = x0
    for y in range(y0, w + 1):
        fill_value(x, y)
    # north side fill
    y = w
    for x in range(w - 1, -w - 1, -1):
        fill_value(x, y)
    # west side fill
    x = -w
    for y in range(w-1, -w-1, -1):
        fill_value(x,y)
    # south side fill
    y = -w
    for x in range(-w + 1, w+1):
        fill_value(x,y)

In [40]:
for p in range(5):
    fill_perimeter(p)
    print(np.max(A), cell)

1.0 312051
25.0 312051
931.0 312051
47108.0 312051
2909666.0 312051


In [41]:
def ck_val(x, y):
    row = ridx(y); col = cidx(x)
    if A[row, col] > cell:
        return A[row, col]
    return 0

def scan_perimeter(p):
    x0, y0 = perimeter_origin(p)
    L = perimeter_side(p)
    w = L//2
    # east side fill
    x = x0
    for y in range(y0, w + 1):
        v = ck_val(x, y)
        if v:
            return v
    # north side fill
    y = w
    for x in range(w - 1, -w - 1, -1):
        v = ck_val(x, y)
        if v:
            return v
    # west side fill
    x = -w
    for y in range(w-1, -w-1, -1):
        v = ck_val(x, y)
        if v:
            return v
    # south side fill
    y = -w
    for x in range(-w + 1, w+1):
        v = ck_val(x, y)
        if v:
            return v
    return 0

In [44]:
scan_perimeter(4)

312453.0